本示例展示如何从晶体结构正向模拟PXRD谱以及如何计算两个谱之间的Rwp loss。掌握本示例的方法之后，你可以:

1. 为自己预测的结构生成模拟谱，并且与A榜的模拟谱进行比对计算Rwp，判断自己预测的结构是否合理. 

2. 从没有PXRD谱的数据集得到带有PXRD谱的数据集，以供后续模型训练. 

我们使用pymatgen库的XRDCalculator进行模拟。pymatgen计算的是理想晶体的谱，输出是一系列独立的峰的位置以及强度$(\theta_i, I_i)$，峰的宽度是0. 实验中得到的谱图峰是有宽度的，为了更接近实验谱，我们需要使用一些峰形函数对pymatgen计算出来的峰进行展宽. 

关于峰形函数的详细内容可以参考：International Tables for Crystallography, Volume H: Powder Diffraction[M]. John Wiley & Sons, 2019. 的1.1.4小节. 

In [ ]:
from pymatgen.core.structure import Structure
from pymatgen.analysis.diffraction.xrd import XRDCalculator
import numpy as np
from tqdm import tqdm

# 定义峰形函数，支持Gaussian, Lorentzian两种
def peak_profile(x, y, FWHM = 0.1, num_points=1000, two_theta_range=(5, 120), shape = "Gaussian"):
    assert shape in ["Gaussian", "Lorentzian"], "shape must be Gaussian or Lorentzian"
    if shape == "Gaussian":
        shape_func = lambda x: np.exp(-4 * np.log(2) * (x / FWHM) ** 2) * (2 * np.sqrt(np.log(2) / np.pi) / FWHM)
    elif shape == "Lorentzian":
        shape_func = lambda x:  FWHM / (FWHM**2/4 + x**2) 
    x_min, x_max = two_theta_range
    x_broad = np.linspace(x_min, x_max, num_points)
    y_broad = np.zeros_like(x_broad)
    for xi, yi in zip(x, y):
        xi_contribute=shape_func(x_broad - xi)
        y_broad += xi_contribute * yi
    
    return x_broad, y_broad

In [ ]:
# 使用XRDCalculator模拟得到离散的谱
# 获取CaCO3结构的cif字符串
import pandas as pd 
file_path = "/bohr/test-uy5f/v2/C/C.csv"
df = pd.read_csv(file_path)
cif_str = df.iloc[0]['cif'] # 这个df 只有一行
st = Structure.from_str(cif_str, fmt='cif')
calc = XRDCalculator()
pattern = calc.get_pattern(st, two_theta_range=(5, 120), scaled = True) 
x = pattern.x
y = pattern.y

# 打印离散的谱
for i in range(len(x)):
    print(f"peak position at {x[i]}, relative intensity:{y[i]}")


In [ ]:
# 使用高斯峰形函数, FWHM参数控制峰的宽度，实际上比赛中使用的谱就是选取FWHM=0.1的高斯峰形函数得到的. 
x_broad, y_broad = peak_profile(x, y, FWHM=0.1, num_points=11501, two_theta_range=(5, 120), shape="Gaussian")
# 可视化
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(x_broad,y_broad)
plt.xlabel("2 theta")
plt.ylabel("intensity")
plt.show()

## Rwp 的计算
对于一个晶体粉末样品，一方面，我们可以实验测量得到其PXRD谱. 具体来说，粉末衍射仪会测量$2 \theta$在某个范围（例如range(5,120, step_size=0.01)）内每个取值上的衍射强度。我们将实验测量得到强度序列记作$y_{obs}$.

另一方面，我们可以用各种方法推断该晶体的结构模型，从结构模型正向模拟计算得到PXRD谱，我们将这个模拟谱记作$y_{calc}$. 

如果结构模型和真实的晶体结构相差很小，那么$y_{calc}$和$y_{obs}$会非常相似。因此我们可以用实验谱和模拟谱的“相似性”来判断结构解析的质量。$R_{wp}$ (Weighed-profile R factor) 是一种常用的相似性指标，其定义如下：
$$ R_{wp} = \left(\sum_j w_j(y_{obs,j}-y_{calc,j})^2/ \sum_j w_j y_{obs,j}^2\right)^{1/2},$$
其中$w_j$是一个权重。一般来说，我们可以设定某个阈值$\epsilon$, 然后设定$w_j = 1/ \max(Y_{obs,j},\epsilon)$. 

下面我们将CaCO3的晶体结构稍微扰动一下，得到扰动结构的谱，计算扰动后的谱和原谱之间的Rwp。 **你可以参照这个逻辑，修改代码，计算你的预测结构的谱和A榜公布的谱之间的Rwp**

In [ ]:
def calculate_rwp(y_calc, y_obs, epsilon=0.01):
    """
    计算加权轮廓R因子 (Rwp)
    
    参数:
    y_calc (numpy.ndarray): 计算得到的衍射强度数组
    y_obs (numpy.ndarray): 实验观测的衍射强度数组
    epsilon (float): 用于防止除零的阈值
    
    返回:
    float: Rwp值
    """
    # 计算权重矩阵
    weights = 1 / np.maximum(y_obs, epsilon)
    
    # 计算分子和分母
    numerator = np.sum(weights * (y_obs - y_calc)**2)
    denominator = np.sum(weights * y_obs**2)
    
    # 计算Rwp值
    rwp = np.sqrt(numerator / denominator)
    return rwp

In [ ]:
from pymatgen.core import PeriodicSite
def perturb_structure(original_structure: Structure, delta: float) -> Structure:
    """
    对结构的分数坐标施加随机扰动
    
    Args:
        original_structure (Structure): 原始结构对象
        delta (float): 扰动幅度（分数坐标范围，例如 0.05 代表±5%的扰动）
        
    Returns:
        Structure: 扰动后的新结构
    """
    lattice = original_structure.lattice  # 获取原始晶格
    perturbed_sites = []  # 存储扰动后的原子位点
    
    for site in original_structure:
        # 生成随机扰动值（均匀分布）
        perturbation = np.random.uniform(-delta, delta, 3)
        # 计算新的分数坐标
        new_frac_coords = site.frac_coords + perturbation
        # 处理周期性边界（确保坐标在 [0,1) 范围内）
        new_frac_coords = np.mod(new_frac_coords, 1.0)
        
        # 创建新的 PeriodicSite 对象
        new_site = PeriodicSite(
            species=site.species,
            coords=new_frac_coords,
            lattice=lattice,
            coords_are_cartesian=False,
            properties=site.properties,  # 可选：保留原属性（如磁性、电荷等）
        )
        perturbed_sites.append(new_site)
    
    # 构建新结构
    return Structure.from_sites(perturbed_sites)

In [ ]:
# 下面我们将原始的CaCO3的结构，扰动其原子坐标，计算扰动后的谱，并计算扰动后的谱和原始的计算谱的Rwp.
perturbed_st = perturb_structure(st,delta = 0.005) # 你可以用你预测的结构来替换这里的扰动结构

pattern_perturbed = calc.get_pattern(perturbed_st, two_theta_range=(5, 120), scaled = True) 
x_broad_perturbed, y_broad_perturbed = peak_profile(pattern_perturbed.x, pattern_perturbed.y, FWHM=0.1, num_points=11501, two_theta_range=(5, 120), shape="Gaussian")

# 画出谱图对比
plt.figure(figsize=(10,6))
plt.plot(x_broad,y_broad,label="original",color="blue")
plt.plot(x_broad_perturbed,y_broad_perturbed, label="perturbed", color="red")
plt.xlabel("2 theta")
plt.ylabel("intensity")
plt.legend()
plt.show()

In [ ]:
#计算Rwp
#!!!!! 请注意，由于这里的强度是相对强度，请将谱图归一化之后再计算Rwp！
y_broad = y_broad /np.max(y_broad)*100
y_broad_perturbed = y_broad_perturbed/np.max(y_broad_perturbed) * 100
Rwp =calculate_rwp(y_broad_perturbed, y_broad)
print(f"The Rwp is {Rwp}") 



Rwp是一个很敏感的指标，**一般只在结构解析的最后阶段：Rietveld精修阶段使用**，在这个阶段，结构模型和真实的晶体结构已经差异很小。在晶体结构解析中，如果我们能够得到Rwp在0.15以下，我们认为结构解析成功了。

反过来，如果结构模型和真实结构差异没那么小，特别是当晶格参数还有误差的时候，这时候得到的Rwp通常会很大，**这不能说明选手对结构的解析是失败的**。 选手可以先作出谱图肉眼对比，或者使用其他的相似度指标来衡量，例如L2 loss, 最优运输loss等等。


## 训练数据集的构建
选手可以利用上述的代码，将任何一个晶体结构数据库，变成带有PXRD谱图的数据库，并用这个数据库作为自己的训练集。本次比赛没有设置训练集，任何训练集都是允许的。

一些晶体结构数据库：
1. Materials Project https://next-gen.materialsproject.org/
2. CoD https://www.crystallography.net/cod/
3. DiffCSP的训练数据集, MP-20, MPTS52....： https://github.com/jiaor17/DiffCSP